In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tables as tb
from memory_profiler import memory_usage

from invisible_cities.cities import components as cp
import invisible_cities.database.load_db

In [29]:
import scipy.stats

In [ ]:
num_files = 101
data_dir = '/Users/taylorcontreras/Development/Research/trackingplane/data/new_sims/'
database = invisible_cities.database.load_db.DataSiPM('flexs13p15')

In [ ]:
h5 = tb.open_file(fast_sims[0])

In [ ]:
def first_nonzero(arr, axis=0, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

In [ ]:
%%time
events = 100
event_id = 0
event_ids, sipm_charge, xval, yval, zval, sensors = [], [], [], [], [], []
for evt in range(0,events):
    sens = 0

    for sipmi in h5.root.sipmrd[evt, :]:
            if(sum(sipmi) > 0.):
                event_ids.append(event_id)
                sipm_charge.append(sum(sipmi))
                sensor_id = 1000+sens
                x = db[db.SensorID == sensor_id].X.values[0]
                y = db[db.SensorID == sensor_id].Y.values[0]
                z = first_nonzero(np.array(sipmi), 0) - 100
                sens = sens + 1
                #print(sens)
                xval.append(x)
                yval.append(y)
                zval.append(z)
                sensors.append(sensor_id)
            else:
                sens = sens + 1

In [ ]:
pmt_charge = np.sum(h5.root.pmtrd, axis=2).flatten()
np.shape(pmt_charge)

In [ ]:
np.shape(h5.root.pmtrd)[1]

In [ ]:
pmt_charge = np.sum(h5.root.pmtrd, axis=2).flatten()
npmts = np.shape(h5.root.pmtrd)[1]
pmt_ids = np.arange(0, npmts)
event_ids = np.arange(0, events) + event_id
event_id = event_ids[-1] + 1
pmt_ids = np.repeat([pmt_ids], repeats=events, axis=0).flatten()
pmt_events = np.repeat(event_ids, npmts)
pmttable = {'event_id': pmt_events,
            'charge': pmt_charge,
            'sensor_id':pmt_ids}
print(pmttable)

In [ ]:
np.shape(h5.root.pmtrd)

In [ ]:
#%%time
summed = np.sum(h5.root.sipmrd, axis=2)
pmt_charge = np.sum(h5.root.pmtrd, axis=2).flatten()
nsipms = np.shape(summed)[1]
npmts = np.shape(h5.root.pmtrd)[1]
this_data = summed.flatten()
sens = 1000
# Make sensor and event id numbers, SiPMs start at 1000
ids = np.arange(sens, sens+nsipms)
event_ids = np.arange(0, events)

# Get sensor positions from the database based on sensor id
xs = db[db.SensorID.isin(ids)].X.values
ys = db[db.SensorID.isin(ids)].Y.values

# Get time (z pos) of event based on sensor time bins
zs = np.apply_along_axis(first_nonzero, axis=2, arr=h5.root.sipmrd).flatten() - 100

# Repeat for the number of events to match size of data
ids = np.repeat([ids], repeats=events, axis=0).flatten()
xs = np.repeat([xs], repeats=events, axis=0).flatten()
ys = np.repeat([ys], repeats=events, axis=0).flatten()
zs = np.repeat([zs], repeats=events, axis=0).flatten()

# Repeat event ids for number of sensors
event_ids = np.arange(0,events)
event_ids = np.repeat(event_ids,nsipms)

# Remove sensors with zero charge in an event
nonzero_entries = np.where(this_data>0)
#ids = ids[nonzero_entries]
#this_data_nz = this_data[nonzero_entries]
#xs = xs[nonzero_entries]
#ys = ys[nonzero_entries]
#zs = zs[nonzero_entries]
#event_ids = event_ids[nonzero_entries]

In [ ]:
np.shape(nonzero_entries)

In [ ]:
np.shape(ids)

In [ ]:
np.shape(this_data)

In [ ]:
xs = xs[nonzero_entries[0]]

In [ ]:
1526800/330800

In [ ]:
plt.plot(event_ids)
plt.show()

In [ ]:
plt.plot(zs)
plt.plot(zval)

In [ ]:
plt.hist(zs)
plt.hist(zval)

In [ ]:
    sipmtable = {'event_id': event_ids,
                 'charge': this_data[nonzero_entries],
                  'X':xs,
                  'Y':ys,
                  'Z':zs,
                  'sensor_id': ids}
    print(sipmtable)

    sipmtable_df = pd.DataFrame(sipmtable)

In [ ]:
sipmtable_df

In [ ]:
%%time
events = 1
event_id = 0
event_ids, pmt_charge, xval, yval, zval, sensors = [], [], [], [], [], []
for evt in range(0,events):
    sens = 0

    for pmti in h5.root.pmtrd[evt, :]:
            if(sum(pmti) > 0.):
                event_ids.append(event_id)
                pmt_charge.append(sum(pmti))
                sensor_id = sens
                #x = db[db.SensorID == sensor_id].X.values[0]
                #y = db[db.SensorID == sensor_id].Y.values[0]
                #z = first_nonzero(np.array(sipmi), 0) - 100
                sens = sens + 1
                #print(sens)
                print(sens)
                print(pmt_charge)
                sensors.append(sensor_id)
            else:
                sens = sens + 1

In [ ]:
np.shape(np.sum(h5.root.pmtrd, axis=2))

# Checking Memory Usage

In [ ]:
def BruteForce():
    events = 100
    event_id = 0
    event_ids, event_pmts, sipm_charge, pmt_charge, xval, yval, zval, sensors, pmt_sensors = [], [], [], [], [], [], [], [], []
        
    for evt in range(0,events):
        sens = 1000

        for sipmi in h5.root.sipmrd[evt, :]:
            charge = sum(sipmi)
            if charge > 0:
                event_ids.append(event_id)
                sipm_charge.append(charge)
                xval.append(database[database.SensorID == sens].X.values[0])
                yval.append(database[database.SensorID == sens].Y.values[0])
                zval.append(first_nonzero(np.array(sipmi), 0) - 100)
                sensors.append(sens)
            sens += 1
                
        sens = 0
        for pmti in h5.root.pmtrd[evt, :]:
            charge = sum(pmti)
            if charge > 0:
                pmt_charge.append(charge)
                pmt_sensors.append(sens)
                event_pmts.append(event_id)
            sens += 1
            
        event_id += 1
            
    sipmtable = {'event_id': event_ids,
             'charge': sipm_charge,
              'X':xval,
              'Y':yval,
              'Z':zval,
              'sensor_id': sensors}

    sipmtable_df = pd.DataFrame(sipmtable)

    pmttable = {'event_id': event_pmts,
                'charge': pmt_charge,
              'sensor_id': pmt_sensors}

    pmttable_df = pd.DataFrame(pmttable)
    
    return sipmtable_df, pmttable_df
                    
def Vectorize():
    events = 100
    sipm_charge = np.sum(h5.root.sipmrd, axis=2)
    pmt_charge = np.sum(h5.root.pmtrd, axis=2).flatten()
    nsipms = np.shape(sipm_charge)[1]
    npmts = np.shape(h5.root.pmtrd)[1]
    sipm_charge = sipm_charge.flatten()
    sens = 1000
    # Make sensor and event id numbers, SiPMs start at 1000
    ids = np.arange(sens, sens+nsipms)
    pmt_ids = np.arange(0,npmts)
    event_ids = np.arange(0, events)

    # Get sensor positions from the database based on sensor id
    xs = database[database.SensorID.isin(ids)].X.values
    ys = database[database.SensorID.isin(ids)].Y.values

    # Get time (z pos) of event based on sensor time bins
    zs = np.apply_along_axis(first_nonzero, axis=2, arr=h5.root.sipmrd).flatten() - 100

    # Repeat for the number of events to match size of data
    ids = np.repeat([ids], repeats=events, axis=0).flatten()
    pmt_ids = np.repeat([pmt_ids], repeats=events, axis=0).flatten()
    xs = np.repeat([xs], repeats=events, axis=0).flatten()
    ys = np.repeat([ys], repeats=events, axis=0).flatten()
    zs = np.repeat([zs], repeats=events, axis=0).flatten()

    # Repeat event ids for number of sensors
    event_ids = np.arange(0,events)
    event_pmts = np.repeat(event_ids, npmts)
    event_ids = np.repeat(event_ids,nsipms)

    # Remove sensors with zero charge in an event
    nonzero_entries = np.where(sipm_charge>0)
    ids = ids[nonzero_entries]
    sipm_charge = sipm_charge[nonzero_entries]
    xs = xs[nonzero_entries]
    ys = ys[nonzero_entries]
    zs = zs[nonzero_entries]
    event_ids = event_ids[nonzero_entries]
    
    sipmtable = {'event_id': event_ids,
             'charge': sipm_charge,
              'X':xs,
              'Y':ys,
              'Z':zs,
              'sensor_id': ids}

    sipmtable_df = pd.DataFrame(sipmtable)

    print(np.shape(event_pmts), np.shape(pmt_charge), np.shape(pmt_ids))
    pmttable = {'event_id': event_pmts,
                'charge': pmt_charge,
              'sensor_id': pmt_ids}

    pmttable_df = pd.DataFrame(pmttable)
    
    return sipmtable_df, pmttable_df
    
def VectorizeLessMem():
    events = 100
    sipm_charge = np.sum(h5.root.sipmrd, axis=2)
    pmt_charge = np.sum(h5.root.pmtrd, axis=2).ravel()
    nsipms = np.shape(sipm_charge)[1]
    npmts = np.shape(h5.root.pmtrd)[1]
    sipm_charge = sipm_charge.ravel()
    sens = 1000
    # Make sensor and event id numbers, SiPMs start at 1000
    ids = np.arange(sens, sens+nsipms)
    pmt_ids = np.arange(0,npmts)
    event_ids = np.arange(0, events)

    # Get sensor positions from the database based on sensor id
    xs = database[database.SensorID.isin(ids)].X.values
    ys = database[database.SensorID.isin(ids)].Y.values

    # Get time (z pos) of event based on sensor time bins
    zs = np.apply_along_axis(first_nonzero, axis=2, arr=h5.root.sipmrd).ravel() - 100

    # Repeat for the number of events to match size of data
    ids = np.repeat([ids], repeats=events, axis=0).ravel()
    pmt_ids = np.repeat([pmt_ids], repeats=events, axis=0).ravel()
    xs = np.repeat([xs], repeats=events, axis=0).ravel()
    ys = np.repeat([ys], repeats=events, axis=0).ravel()
    zs = np.repeat([zs], repeats=events, axis=0).ravel()

    # Repeat event ids for number of sensors
    event_ids = np.arange(0,events)
    event_pmts = np.repeat(event_ids, npmts)
    event_ids = np.repeat(event_ids,nsipms)

    # Remove sensors with zero charge in an event
    nonzero_entries = np.where(sipm_charge>0)
    ids = ids[nonzero_entries]
    sipm_charge = sipm_charge[nonzero_entries]
    xs = xs[nonzero_entries]
    ys = ys[nonzero_entries]
    zs = zs[nonzero_entries]
    event_ids = event_ids[nonzero_entries]
    
    sipmtable = {'event_id': event_ids,
             'charge': sipm_charge,
              'X':xs,
              'Y':ys,
              'Z':zs,
              'sensor_id': ids}

    sipmtable_df = pd.DataFrame(sipmtable)

    print(np.shape(event_pmts), np.shape(pmt_charge), np.shape(pmt_ids))
    pmttable = {'event_id': event_pmts,
                'charge': pmt_charge,
              'sensor_id': pmt_ids}

    pmttable_df = pd.DataFrame(pmttable)
    
    return sipmtable_df, pmttable_df
    
def VectorizePerEvent(sipm_event, pmt_event, event_id, database):
    sipm_charge = np.sum(sipm_event, axis=1)
    pmt_charge = np.sum(pmt_event, axis=1) 
    nsipms = np.shape(sipm_charge)[0]
    npmts = np.shape(pmt_charge)[0]
    sipm_charge = sipm_charge.ravel()
    pmt_charge = pmt_charge.ravel()
    sens = 1000
    # Make sensor and event id numbers, SiPMs start at 1000
    ids = np.arange(sens, sens+nsipms)
    pmt_ids = np.arange(0, npmts)

    # Get sensor positions from the database based on sensor id
    xs = database[database.SensorID.isin(ids)].X.values
    ys = database[database.SensorID.isin(ids)].Y.values

    # Get time (z pos) of event based on sensor time bins
    zs = np.apply_along_axis(first_nonzero, axis=1, arr=sipm_event).ravel() - 100

    # Repeat event ids for number of sensors
    event_ids = [event_id]
    pmt_events = np.repeat(event_ids, npmts)
    event_ids = np.repeat(event_ids, nsipms)

    # Remove sensors with zero charge in an event
    nonzero_entries = np.where(sipm_charge>0)
    ids = ids[nonzero_entries]
    sipm_charge = sipm_charge[nonzero_entries]
    xs = xs[nonzero_entries]
    ys = ys[nonzero_entries]
    zs = zs[nonzero_entries]
    event_ids = event_ids[nonzero_entries]
    
    return event_ids, sipm_charge, xs, ys, zs, ids, pmt_events, pmt_charge, pmt_ids
    
def VectorizeEvents(h5, database):
    nevents = 100
    event_ids = np.arange(0, nevents)
    sipm_events = h5.root.sipmrd
    pmt_events = h5.root.pmtrd

    event_ids, sipm_charge, xs, ys, zs, ids, pmt_eids, pmt_charge, pmt_ids = [], [], [], [], [], [], [], [], []
    for event in range(0,nevents):
        sipm_data = sipm_events[event]
        pmt_data = pmt_events[event]
        events, s_charge, x, y, z, i, pmt_event, pmt_c, pmt_id = VectorizePerEvent(sipm_data, pmt_data, event, database)
        event_ids.append(events)
        sipm_charge.append(s_charge)
        xs.append(x)
        ys.append(y)
        zs.append(z)
        ids.append(i)
        pmt_eids.append(pmt_event)
        pmt_charge.append(pmt_c)
        pmt_ids.append(pmt_id)
        
    sipmtable = {'event_id': event_ids,
         'charge': sipm_charge,
          'X':xs,
          'Y':ys,
          'Z':zs,
          'sensor_id': ids}

    sipmtable_df = pd.DataFrame(sipmtable)

    pmttable = {'event_id': pmt_eids,
                'charge': pmt_charge,
              'sensor_id': pmt_ids}

    return sipmtable_df, pmttable_df
        
def VectorizePerSet(event_set, event_id):
    summed = np.sum(event_set, axis=1)
    #pmt_charge = np.sum(h5.root.pmtrd, axis=2).flatten()
    nsipms = np.shape(summed)[0]
    #npmts = np.shape(h5.root.pmtrd)[1]
    summed = summed.ravel()
    sens = 1000
    # Make sensor and event id numbers, SiPMs start at 1000
    ids = np.arange(sens, sens+nsipms)
    event_ids = np.arange(0, events)

    # Get sensor positions from the database based on sensor id
    xs = database[database.SensorID.isin(ids)].X.values
    ys = database[database.SensorID.isin(ids)].Y.values

    # Get time (z pos) of event based on sensor time bins
    zs = np.apply_along_axis(first_nonzero, axis=2, arr=h5.root.sipmrd).ravel() - 100

    # Repeat for the number of events to match size of data
    ids = np.repeat([ids], repeats=events, axis=0).ravel()
    xs = np.repeat([xs], repeats=events, axis=0).ravel()
    ys = np.repeat([ys], repeats=events, axis=0).ravel()
    zs = np.repeat([zs], repeats=events, axis=0).ravel()

    # Repeat event ids for number of sensors
    event_ids = np.arange(0,events)
    event_ids = np.repeat(event_ids,nsipms)

    # Remove sensors with zero charge in an event
    nonzero_entries = np.where(summed>0)
    ids = ids[nonzero_entries]
    summed = summed[nonzero_entries]
    xs = xs[nonzero_entries]
    ys = ys[nonzero_entries]
    zs = zs[nonzero_entries]
    event_ids = event_ids[nonzero_entries]
    
def VectorizeAll():
    nevents = 100
    event_ids = np.arange(0, nevents)
    events = h5.root.sipmrd
    group_sipms = 1000
    event = h5.root.sipmrd[0]
    nsipms = np.shape(event)[0]
    start_sipms = np.arange(0,nsipms, group_sipms)
    for event, event_id in zip(events, event_ids):
        for start in start_sipms:
            VectorizePerEvent(event[start:start+group_sipms], event_id)

In [ ]:
%%time
sipm_data, pmt_data = VectorizeAll()

In [ ]:
%%time
sipm_data, pmt_data = VectorizeEvents(h5, database)

In [ ]:
%%time
sipm_data, pmt_data = Vectorize()

In [ ]:
%%time
sipm_data, pmt_data = BruteForce()

In [ ]:
mem_usage = memory_usage(BruteForce)
print('Maximum memory usage: %s' % max(mem_usage))

In [ ]:
mem_usage = memory_usage(Vectorize)
print('Maximum memory usage: %s' % max(mem_usage))

In [ ]:
mem_usage = memory_usage(VectorizeEvents(h5, database))
print('Maximum memory usage: %s' % max(mem_usage))

In [ ]:
mem_usage = memory_usage(VectorizeAll)
#print('Memory usage (in chunks of .1 seconds): %s' % mem_usage)
print('Maximum memory usage: %s' % max(mem_usage))

In [ ]:
# From tests on cluster iwth s1.3mmp15mm sim
funcs_time = [44, 17, 22, 22, 60*2+45]
funcs_mem = [1.36*1000, 5.8, 5.8, 6.23, 142.75]
funcs = ['VectorizeAll', 'VectorizeEvents', 'VectorizeLessMem', 'Vectorize', 'BruteForce']
for f in range(0,len(funcs)):
    plt.plot(funcs_time[f], funcs_mem[f], 'o', label=funcs[f])
plt.xlabel('Run time')
plt.ylabel('Memory used [MB]')
plt.legend()
plt.show()
for f in range(1,len(funcs)-1):
    plt.plot(funcs_time[f], funcs_mem[f], 'o', label=funcs[f])
plt.xlabel('Run time')
plt.ylabel('Memory used [MB]')
plt.legend()
plt.show()

# Fixing VectorizeEvents

In [ ]:
def VectorizePerEvent(sipm_event, pmt_event, event_id, database):
    sipm_charge = np.sum(sipm_event, axis=1)
    pmt_charge = np.sum(pmt_event, axis=1) 
    nsipms = np.shape(sipm_charge)[0]
    npmts = np.shape(pmt_charge)[0]
    sipm_charge = sipm_charge.ravel()
    pmt_charge = pmt_charge.ravel()
    sens = 1000
    # Make sensor and event id numbers, SiPMs start at 1000
    ids = np.arange(sens, sens+nsipms)
    pmt_ids = np.arange(0, npmts)

    # Get sensor positions from the database based on sensor id
    xs = database[database.SensorID.isin(ids)].X.values
    ys = database[database.SensorID.isin(ids)].Y.values

    # Get time (z pos) of event based on sensor time bins
    zs = np.apply_along_axis(first_nonzero, axis=1, arr=sipm_event).ravel() - 100

    # Repeat event ids for number of sensors
    event_ids = [event_id]
    pmt_events = np.repeat(event_ids, npmts)
    event_ids = np.repeat(event_ids, nsipms)

    # Remove sensors with zero charge in an event
    nonzero_entries = np.where(sipm_charge>0)
    ids = np.array(ids[nonzero_entries])
    sipm_charge = np.array(sipm_charge[nonzero_entries])
    xs = np.array(xs[nonzero_entries])
    ys = np.array(ys[nonzero_entries])
    zs = np.array(zs[nonzero_entries])
    event_ids = np.array(event_ids[nonzero_entries])
    
    return event_ids, sipm_charge, xs, ys, zs, ids, pmt_events, pmt_charge, pmt_ids
    
def VectorizeEvents(h5, database):
    nevents = 100
    event_ids = np.arange(0, nevents)
    sipm_events = h5.root.sipmrd
    pmt_events = h5.root.pmtrd

    event_ids, sipm_charge, xs, ys, zs, ids, pmt_eids, pmt_charge, pmt_ids = np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])
    for event in range(0,nevents):
        sipm_data = sipm_events[event]
        pmt_data = pmt_events[event]
        events, s_charge, x, y, z, i, pmt_event, pmt_c, pmt_id = VectorizePerEvent(sipm_data, pmt_data, event, database)
        event_ids = np.concatenate((event_ids,events))
        sipm_charge = np.concatenate((sipm_charge,s_charge))
        xs = np.concatenate((xs,x))
        ys = np.concatenate((ys,y))
        zs = np.concatenate((zs,z))
        ids = np.concatenate((ids,i))
        pmt_eids = np.concatenate((pmt_eids,pmt_event))
        pmt_charge = np.concatenate((pmt_charge,pmt_c))
        pmt_ids = np.concatenate((pmt_ids,pmt_id))
        
    sipmtable = {'event_id': event_ids,
         'charge': sipm_charge,
          'X':xs,
          'Y':ys,
          'Z':zs,
          'sensor_id': ids}

    sipmtable_df = pd.DataFrame(sipmtable)

    pmttable = {'event_id': pmt_eids,
                'charge': pmt_charge,
              'sensor_id': pmt_ids}

    return sipmtable_df, pmttable_df

In [ ]:
import invisible_cities.database.load_db as db
sipmtable_df, pmttable_df = VectorizeEvents(h5, db.DataSiPM('s13mmp15mm'))

In [ ]:
plt.hist(sipmtable_df.X)
plt.show()

plt.hist(sipmtable_df.Y)
plt.show()

plt.hist(sipmtable_df.Z)
plt.show()

In [25]:
import numpy  as np
import tables as tb
import pandas as pd
import sys

from invisible_cities.io.dst_io import df_writer
import invisible_cities.database.load_db as db
import invisible_cities.io.mcinfo_io as mcio

def first_nonzero(arr, axis=0, invalid_val=-1):
    # Function to find the first nonzero entry
    # in a given array, returning invalid_val if none
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

def GetMaxR(particles):
    # Get r_max position based on true particles position
    return np.sqrt(particles.initial_x**2 + particles.initial_y**2).max()

def VectorizePerEvent(sipm_event, pmt_event, event_id, database):
    sipm_charge = np.sum(sipm_event, axis=1)
    pmt_charge = np.sum(pmt_event, axis=1) 
    nsipms = np.shape(sipm_charge)[0]
    npmts = np.shape(pmt_charge)[0]
    sipm_charge = sipm_charge.ravel()
    pmt_charge = pmt_charge.ravel()
    sens = 1000
    # Make sensor and event id numbers, SiPMs start at 1000
    ids = np.arange(sens, sens+nsipms)
    pmt_ids = np.arange(0, npmts)

    # Get sensor positions from the database based on sensor id
    xs = database[database.SensorID.isin(ids)].X.values
    ys = database[database.SensorID.isin(ids)].Y.values
    
    # Get time (z pos) of event based on sensor time bins
    zs = np.apply_along_axis(first_nonzero, axis=1, arr=sipm_event).ravel() - 100

    # Repeat event ids for number of sensors
    event_ids = [event_id]
    pmt_events = np.repeat(event_ids, npmts)
    event_ids = np.repeat(event_ids, nsipms)

    # Remove sensors with zero charge in an event
    nonzero_entries = np.where(sipm_charge>0)
    ids = np.array(ids[nonzero_entries])
    sipm_charge = np.array(sipm_charge[nonzero_entries])
    xs = np.array(xs[nonzero_entries])
    ys = np.array(ys[nonzero_entries])
    zs = np.array(zs[nonzero_entries])
    event_ids = np.array(event_ids[nonzero_entries])
    
    return event_ids, sipm_charge, xs, ys, zs, ids, pmt_events, pmt_charge, pmt_ids
    
def ReduceWaveforms(h5file, outname, dbname, fnum, nevents=100):
    event_start = fnum * nevents
    h5 = tb.open_file(h5file)
    database = db.DataSiPM(dbname)
    sipm_events = h5.root.sipmrd
    pmt_events = h5.root.pmtrd
    
    particles = mcio.load_mcparticles_df(h5file).reset_index()
    rs = particles.groupby('event_id').apply(lambda grp: np.sqrt(grp.initial_x**2 + grp.initial_y**2).max())

    event_ids, sipm_charge, xs, ys, zs, ids, pmt_eids, pmt_charge, pmt_ids = np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])
    for event in range(0,nevents):
        print('Event: ',event)
        sipm_data = sipm_events[event]
        pmt_data = pmt_events[event]
        events, s_charge, x, y, z, i, pmt_event, pmt_c, pmt_id = VectorizePerEvent(sipm_data, pmt_data, event, database)
        
        events = events + event_start
        event_ids = np.concatenate((event_ids,events))
        sipm_charge = np.concatenate((sipm_charge,s_charge))
        xs = np.concatenate((xs,x))
        ys = np.concatenate((ys,y))
        zs = np.concatenate((zs,z))
        ids = np.concatenate((ids,i))
        pmt_eids = pmt_eids + event_start
        pmt_eids = np.concatenate((pmt_eids,pmt_event))
        pmt_charge = np.concatenate((pmt_charge,pmt_c))
        pmt_ids = np.concatenate((pmt_ids,pmt_id))
    
    print('Event_ids', event_ids)
    sipmtable = {'event_id': event_ids,
         'charge': sipm_charge,
          'X':xs,
          'Y':ys,
          'Z':zs,
          'sensor_id': ids}
    sipmtable_df = pd.DataFrame(sipmtable)

    pmttable = {'event_id': pmt_eids,
                'charge': pmt_charge,
              'sensor_id': pmt_ids}
    pmttable_df = pd.DataFrame(pmttable)
    
    rtable = {'r_max':rs, 'event_id':particles.event_id.unique()}
    rtable_df = pd.DataFrame(rtable)

    print(f'Out {outname}')
    h5out = tb.open_file(outname, 'w')

    df_writer(h5out, sipmtable_df, 'SiPM',  'Waveforms')
    df_writer(h5out, pmttable_df, 'PMT', 'Waveforms')
    df_writer(h5out, rtable_df, 'particles', 'r_max')

    h5out.close()

    return

In [7]:
import invisible_cities.io.mcinfo_io as mcio
particles = mcio.load_mcparticles_df(h5file).reset_index()
#evtmap = mcio.load_eventnumbermap(h5file).set_index('nexus_evt')
#particles.event_id = particles.event_id.map(evtmap.evt_number)
particles = particles[particles.primary==True]

In [ ]:
event = particles[particles.event_id == 0]
r = np.sqrt(event.initial_x**2 + event.initial_y**2)
r_max = r.max()
r_max

In [ ]:
r_max = particles.groupby('event_id').apply(lambda grp: np.sqrt(grp.initial_x**2 + grp.initial_y**2).max())

In [8]:
particles.event_id.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [2]:
data_dir = '/Users/taylorcontreras/Development/Research/trackingplane/data/test_12082022/'
h5file = data_dir + 'flex.0vbb.0.buffy.s1.3mmp2.4mm.h5'
outname = data_dir + 'test.h5'
dbname = 's13mmp24mm'
fnum = 999
nevents = 10

In [3]:
h5file

'/Users/taylorcontreras/Development/Research/trackingplane/data/test_12082022/flex.0vbb.0.buffy.s1.3mmp2.4mm.h5'

In [26]:
ReduceWaveforms(h5file, outname, dbname, fnum, nevents=10)

/Users/taylorcontreras/anaconda3/envs/IC-3.7-2020-06-16/lib/python3.7/site-packages/tables/leaf.py:410: PerformanceWarning: The Leaf ``/sipmrd`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


Event:  0
Event:  1
Event:  2
Event:  3
Event:  4
Event:  5
Event:  6
Event:  7
Event:  8
Event:  9
Event_ids [9990. 9990. 9990. ... 9999. 9999. 9999.]
Out /Users/taylorcontreras/Development/Research/trackingplane/data/test_12082022/test.h5


In [23]:
test_out = pd.read_hdf(outname, 'particles/r_max')

In [24]:
test_out

,r_max,event_id
0,337.972107,0
1,430.106934,1
2,477.085541,2
3,391.834381,3
4,314.805878,4
5,481.418823,5
6,562.020203,6
7,371.372437,7
8,305.493134,8
9,559.332153,9
